In [2]:
import os
import subprocess
import numpy as np
import cv2
import shutil

In [26]:
dirlist = subprocess.check_output(["find", "./Löwenzahn/Plastik Modell/ohne/"])

In [27]:
images = [e for e in dirlist.decode('utf-8').splitlines() if e[-3:].lower() in ["jpg","png"]]

In [28]:
names = [f.lower().replace(":","_").replace(" ","_").replace("/markiert","").split("/")[-1] for f in images]

In [29]:
filenames_labels = dict()
filenames_images = dict()
file = open("./X_public_jpgs.txt",'r')
approved_imgs = file.read().replace(".jpg\n", ", ").split(", ")

In [30]:

for name, image in zip(names, images):
    if name.replace(".png", "").replace(".jpg", "") in approved_imgs:
        if name.endswith("jpg"):
            filenames_images[name[:-4]] = image
        elif name.endswith("png"):
            filenames_labels[name[:-4]] = image
        else:
            raise ValueError("Invalid filename "+name+" for image "+image)

In [31]:
keys = list(filenames_labels.keys())

In [32]:
MARKER_DANDELION = np.array([255,255,0])
MARKER_ALCOHOLIC = np.array([0,255,255])

def load_label(name):
    raw = cv2.imread(filenames_labels[name])[:,:,::-1]
    mask_dandelion = (np.abs(raw - MARKER_DANDELION).mean(axis=-1) < 20 ).astype(np.uint8)
    mask_alcoholic = (np.abs(raw - MARKER_ALCOHOLIC).mean(axis=-1) < 20 ).astype(np.uint8)
    mask_none = np.ones_like(mask_dandelion) - mask_dandelion - mask_alcoholic
    return np.stack([mask_dandelion,mask_alcoholic, mask_none], axis=-1) * 255

In [33]:
for key in keys:
    out_image = key + ".jpg"
    out_label = key + ".png"
    
    if not os.path.isfile(out_image):
        print("copying image", out_image)
        shutil.copyfile(filenames_images[key], out_image)
    else:
        print("skipping image ", out_image)
        
    if not os.path.isfile(out_label):
        print("copying label", out_label)
        label = load_label(key)
        cv2.imwrite(out_label, label)
    else:
        print("skipping label", out_label)

In [4]:
def convert_to_yolo_coco(dir: str, dest: str):
    os.mkdir(dest)
    for img in os.listdir(os.path.join(dir, "images")):
        new_img_dir = os.path.join(dest, img.replace(".jpg", ""))
        os.mkdir(new_img_dir)
        os.mkdir(os.path.join(new_img_dir, "images"))
        os.mkdir(os.path.join(new_img_dir, "masks"))
        shutil.copy(os.path.join(dir, "images", img), os.path.join(new_img_dir, "images"))
        
        label = cv2.imread(os.path.join(dir, "labels", img.replace(".jpg", ".png")))[:,:,::-1]
        # plants
        cv2.imwrite(os.path.join(new_img_dir, "masks", img.replace(".jpg", "_class_1.png")), label[:,:,2])
        # cans
        cv2.imwrite(os.path.join(new_img_dir, "masks", img.replace(".jpg", "_class_2.png")), label[:,:,1])


        #shutil.copy(os.path.join(dir, "labels", img.replace(".jpg", ".png")), os.path.join(new_img_dir, "masks"))
convert_to_yolo_coco("./dataset/", "./coco/")

In [5]:
shutil.rmtree("./coco/")
